same as s20220202_0dicom2nrrd4metadata, except list of patientIDs is specified in the script, instead of through the metadata file provided.

In [1]:
import pickle
import pandas as pd
import numpy as np
import nrrd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from IPython.display import clear_output

from radiomics import featureextractor, getTestCase
import SimpleITK as sitk
import subprocess

import os
import glob
import shutil

import time
import datetime

print(datetime.datetime.now())

2021-12-27 11:43:05.447164


In [2]:
path_data = 'data/d20211203/training/'

# INPUTS
fn_patient = 'finalPatientList_20211220.xlsx'
path_masks = path_data + '0raw_masks/'
path_dcmzips = path_data + '0raw_dicom/'

# TEMP PATHS
temp_unzip = path_data + 'temp_unzip/'

# OUTPUTS
path_nrrds = path_data + 'nrrds_metadata/'

In [3]:
t0 = time.time()

listMasks = ['1007', '1009', '1010', '1011', '1012', '1013', '1014', '1015', '1016', '1017', '1019', '1020', '1022', '1027', '1029', '1030', '1031', '1033', '1034', '1035', '1037', '1044', '1047', '1053', '1054', '1056', '1057', '1060', '1064', '1065', '1066', '1068', '1069', '1070', '1072', '1073', '1074', '1075', '1076', '1078', '1079', '1080', '1081', '1082', '1083', '1084', '1085', '1086', '1087', '1088', '1089', '1090', '1091', '1093', '1094', '1095', '1096', '1097', '1098', '1099', '1104', '1109', '1117', '1118', '1121', '1122', '1124', '1126', '1128', '1130', '1133', '1134', '1136']
listConvSuccess = []
listNoDcm = []
listConvError = []
unzipError = []
checkTree = []

# for each available mask, check for raw image zip
for eaPatient in listMasks:
    patientID = eaPatient
    fn_dcmzip = path_dcmzips + str(patientID) + '.zip'
    if os.path.exists(fn_dcmzip):
        path_nrrdTarget = path_nrrds + str(patientID) + '.nrrd'
        os.mkdir(temp_unzip)
        try:
            t1 = time.time()
            # unzip raw image file
            cmd = 'unzip %s -d %s' %(fn_dcmzip, temp_unzip)   # unzip WHAT -d WHERE
            subprocess.run(cmd, shell=True)
            try:
                # identify signs of hidden folders
                garbageIndicators = ["__MACOSX", "/."]

                # use os.walk to get names of all directories
                listDir = []
                for root, dirs, files in os.walk(temp_unzip, topdown=False):
                    for name in [x for x in dirs]:
                        listDir.append(os.path.join(root, name))
                # remove directories that are hidden folders
                notHiddenDir = [s for s in listDir if not any(x in s for x in garbageIndicators)]
                # check if dicom files are in deepest directory
                potentialDir = max(notHiddenDir)
                potentialFiles = [s for s in os.listdir(potentialDir) if '.dcm' in s and not s.startswith('.')]
                if potentialFiles: # if potentialFiles is not empty
                    path_dcm = potentialDir
                    # convert dicom to nrrd
                    cmd = "plastimatch convert --input %s --output-img %s"%(path_dcm, path_nrrdTarget)
                    output = subprocess.run(cmd, shell=True, capture_output=True)
                    if not os.path.exists(path_nrrdTarget):
                        print(patientID + ': not successful ' + str(time.time() - t1) + ' seconds.')
                        print(output)
                        listConvError.append(patientID)
                    else:
                        print(patientID + ': converted, ' + str(time.time() - t1) + ' seconds.')
                        listConvSuccess.append(patientID)
                else:
                    checkTree.append()
            except:
                print(patientID + ': nrrd conversion error')
                print(cmd)
                listConvError.append(patientID)
        except:
            print(patientID + ': unzip error')
            unzipError.append(patientID)
        # remove temp unzip
        shutil.rmtree(temp_unzip)
    else:
        print(patientID + ': dcm files missing')
        listNoDcm.append(patientID)

1007: converted, 42.249943256378174 seconds.
1009: converted, 29.469552516937256 seconds.
1010: converted, 49.24954581260681 seconds.
1011: converted, 49.93803262710571 seconds.
1012: converted, 53.72664403915405 seconds.
1013: converted, 37.422226428985596 seconds.
1014: converted, 33.54848790168762 seconds.
1015: converted, 56.194701194763184 seconds.
1016: converted, 40.314473152160645 seconds.
1017: converted, 43.150880336761475 seconds.
1019: converted, 40.72467565536499 seconds.
1020: converted, 33.45247983932495 seconds.
1022: converted, 43.24328684806824 seconds.
1027: converted, 38.22276282310486 seconds.
1029: converted, 45.5203275680542 seconds.
1030: converted, 46.58263969421387 seconds.
1031: converted, 42.36615800857544 seconds.
1033: converted, 48.030763149261475 seconds.
1034: converted, 39.96883821487427 seconds.
1035: converted, 49.090473890304565 seconds.
1037: converted, 27.83391046524048 seconds.
1044: converted, 41.46133875846863 seconds.
1047: converted, 46.64134

In [4]:
print('Problem with conversion')
print(listConvError)

print('No raw files:')
print(listNoDcm)

Problem with conversion
[]
No raw files:
[]


In [5]:
# # identify signs of hidden folders
# garbageIndicators = ["__MACOSX", "/."]

# # use os.walk to get names of all directories
# listDir = []
# for root, dirs, files in os.walk(temp_unzip, topdown=False):
#     for name in [x for x in dirs]:
#         listDir.append(os.path.join(root, name))
# # remove directories that are hidden folders
# notHiddenDir = [s for s in listDir if not any(x in s for x in garbageIndicators)]
# # check if dicom files are in deepest directory
# potentialDir = max(notHiddenDir)
# potentialFiles = [s for s in os.listdir(potentialDir) if '.dcm' in s and not s.startswith('.')]

# if potentialFiles: # if potentialFiles is not empty
#     path_dcm = potentialDir
# else:
#     stillStillStillConvError.append()

In [6]:
print(datetime.datetime.now())

2021-12-27 12:35:24.871731
